# 05-2 LLaMA-2

<table align="left"><tr><td>
<a href="https://colab.research.google.com/github/rickiepark/hm-dl/blob/main/05-2-llama2-keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="코랩에서 실행하기"/></a>
</td></tr></table>

코랩에서 이 노트북을 실행하려면 High-RAM CPU 런타임을 사용해야 합니다.

## 케라스에서 LLaMA-2 사용하기

** 라마-2, 라마-3 모델을 사용하려면 먼저 메타에 사용 허가를 얻어야 합니다. 자세한 내용은 도서를 참고하세요.**

캐글에서 라마-2 모델을 다운로드하려면 캐글 API 토큰을 생성하여 ~/.kaggle/ 디렉토리에 저장하세요.

In [1]:
!mkdir ~/.kaggle/
!mv kaggle.json ~/.kaggle/

이 절의 코드를 실행하려면 `keras-nlp` 패키지를 설치해야 합니다.

In [3]:
import keras_nlp

In [4]:
llama2 = keras_nlp.models.LlamaCausalLM.from_preset('llama2_7b_en')
llama2.summary()

100%|██████████| 604/604 [00:00<00:00, 528kB/s]


100%|██████████| 12.6G/12.6G [01:37<00:00, 138MB/s]


100%|██████████| 397/397 [00:00<00:00, 778kB/s]


100%|██████████| 488k/488k [00:00<00:00, 82.8MB/s]


Preprocessor: "llama_causal_lm_preprocessor"

...

Model: "llama_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ llama_backbone_1              │ (None, None, 4096)        │   6,738,415,616 │ padding_mask[0][0],        │
│ (LlamaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 32000)       │     262,144,000 │ llama_backbone_1[0][0]     │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 6,738,415,616 (25.10 GB)

 Trainable params: 6,738,415,616 (25.10 GB)

 Non-trainable params: 0 (0.00 B)

In [5]:
sampler = keras_nlp.samplers.TopPSampler(p=0.8, seed=42)
llama2.compile(sampler=sampler)
llama2.generate('stay hungry, stay', max_length=20)

'stay hungry, stay foolish by Steve Jobs'

## 센텐스피스 토크나이저

In [6]:
llama_tokenizer = keras_nlp.models.LlamaTokenizer.from_preset('llama2_7b_en')

In [7]:
token_ids = llama_tokenizer.tokenize('stay hungry, stay')
token_ids

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([ 7952,  9074, 14793, 29892,  7952], dtype=int32)>

In [8]:
for ids in token_ids:
    print(llama_tokenizer.id_to_token(ids), end=' ')

▁stay ▁hun gry , ▁stay 

In [9]:
llama_tokenizer.tokenize('Hello hello')

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([15043, 22172], dtype=int32)>

In [10]:
llama_tokenizer.detokenize(token_ids)

'stay hungry, stay'